In [15]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv('./data/405957090_5천만건(hum,voc포함).csv')

# df["updated"] = pd.to_datetime(df["updated"])
# df=df.sort_values("updated")

# # -1이 있는 행을 삭제
# df = df.replace(-1, np.nan)  # -1을 NaN으로 변경
# df = df.dropna()  # NaN이 있는 행 삭제

# # 온도가 1500 이상 4500 이하인 데이터만 선택
# df = df[(df['temperature'] >= 1500) & (df['temperature'] <= 4500)]

# # 날짜 정상 데이터 선택
# df = df[(df['days'] > 0) & (df['days'] <= 1800)]

# # 성별 정상 데이터 선택
# df = df[df["sex"]!=-1]


# d1=[2179,1766,2220,1970,1702,2411,357,2552,2273,1416]

# df = df[df['device_id'].isin(d1)]

In [16]:
# age_min = df['days'].min()
# age_max = df['days'].max()

# min_device_id = df[df['days']==age_min]['device_id'].iloc[0]
# max_device_id = df[df['days']==age_max]['device_id'].iloc[0]

# min_device_id, max_device_id

(2220, 1416)

In [3]:
import pandas as pd




In [22]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

df_357 = pd.read_csv('./data/senhub/hub_device_357.csv')

def  make_df(_df):
    
    # 데이터 전처리
    _df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)
    _df['updated'] = pd.to_datetime(_df['updated'])
    _df.rename(columns={'device_id_x' : 'device_id'}, inplace=True)
        
    # 날짜 리스트 설정
    day_list = _df['updated'].dt.date.unique()
    
    # 평균값 담을 객체 생성
    sensor_mean_list = [0] * 3
    hub_mean_list = [0] * 3

    # 날짜별 데이터 호출
    for day in day_list:
        day_df = _df[_df['updated'].dt.date == day]
        
        # 해당 날짜의 데이터가 없다면 continue를 사용하여 다음 반복으로 넘어갑니다.
        if day_df['temperature'].isna().all():
            continue
        
        # 센서 온도 있는 경우에만 뒤의 값으로 보간법 시행
        day_df['tem_bfill'] = day_df['tem'].bfill()
        # day_df.loc[day_df['temperature'].isnull(), 'tem_bfill'] = None
        
        # 3일 평균 데이터 갱신
        sensor_mean_list.pop(0)
        sensor_mean_list.append(day_df['temperature'].mean())
        
        if not day_df['tem_bfill'].isna().all():
            hub_mean_list.pop(0)
            hub_mean_list.append(day_df['tem_bfill'].mean())
        
        # 3일 평균 데이터 가져오기
        sensor_not_zero = [i for i in sensor_mean_list if i!=0]
        hub_not_zero = [i for i in hub_mean_list if i!=0]
        
        sensor_mean = np.mean(sensor_not_zero)
        hub_mean = np.mean(hub_not_zero)
        
        # 평균 데이터 컬럼 추가
        day_df['sensor_mean'] = sensor_mean * len(day_df)
        day_df['hub_mean'] = hub_mean * len(day_df)
                
        day_df.drop(['device_id', 'days','sex','temperature','humidity','voc','hum','tem'], axis=1, inplace=True)
        
        # 원본 데이터와 concat
        _df = pd.merge(_df, day_df, how='left', on=['updated'])
        
    # 인덱스 설정
    _df.set_index('updated', inplace=True)
    
    # 이동평균 컬럼 추가
    _df['temperature_moving_avg'] = _df['temperature'].rolling(window=1200).mean()
    _df['tem_moving_avg'] = _df['tem_bfill'].rolling(window=1200).mean()
        
    return _df
    
df_357_after = make_df(df_357)
df_357_after

C:\Users\yeong\AppData\Local\Temp\ipykernel_8648\3106642965.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['tem_bfill'] = day_df['tem'].bfill()
c:\Users\yeong\miniconda3\envs\monit\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\yeong\miniconda3\envs\monit\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\yeong\AppData\Local\Temp\ipykernel_8648\3106642965.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

MemoryError: Unable to allocate 4.58 GiB for an array with shape (38, 16188570) and data type float64

In [19]:
df_357_after['sensor_mean'].unique()

array([1.78055339e+08,            nan])

In [ ]:
def draw_plot(_df):
    # # 데이터의 최소값과 최대값으로 y축 고정
    # ymin = _df['temperature'].min()
    # ymax = _df['temperature'].max()

    # 시각화 날짜 리스트 설정
    day_list = _df['updated'].dt.date.unique()

    # 평균값 담을 객체 생성
    sensor_mean_list = [0] * 3
    hub_mean_list = [0] * 3

    # 날짜별 데이터 시각화
    for min_day in day_list:
        day_df = _df[_df['updated'].dt.date == min_day]
        
        # 센서 온도 있는 경우에만 뒤의 값으로 보간법 시행
        day_df['tem'] = day_df['tem'].bfill()
        # day_df.loc[day_df['temperature'].isnull(), 'tem'] = None
        
        # 해당 날짜의 데이터가 없다면 continue를 사용하여 다음 반복으로 넘어갑니다.
        if day_df['temperature'].isna().all():
            continue
        
        # 3일 평균 데이터 갱신
        sensor_mean_list.pop(0)
        sensor_mean_list.append(day_df['temperature'].mean())
        
        if not day_df['tem'].isna().all():
            hub_mean_list.pop(0)
            hub_mean_list.append(day_df['tem'].mean())
        
        # 3일 평균 데이터 가져오기
        sensor_not_zero = [i for i in sensor_mean_list if i!=0]
        hub_not_zero = [i for i in hub_mean_list if i!=0]
        
        sensor_mean = np.mean(sensor_not_zero)
        hub_mean = np.mean(hub_not_zero)
        
        
        # 그래프 시각화
        fig, ax = plt.subplots(figsize=(10,5))
        
        # 온도 그래프
        ax.plot(day_df['updated'], day_df['temperature'], color='tab:red', label='sensor')
        
        if not day_df['tem'].isna().all():
            ax.plot(day_df['updated'], day_df['tem'], color='tab:green', label='hub')
        
        # 평균값 그래프
        ax.axhline(y=sensor_mean, color='tab:blue', label='sensor_mean')
        
        if len(hub_not_zero) != 0:
            ax.axhline(y=hub_mean, color='tab:purple', label='hub_mean')
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Temperature')
        ax.tick_params(axis='y')
        ax.legend()
        
        # y축 범위 설정
        # ax.set_ylim(ymin, ymax)

        # 시간 형식 설정
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        
        fig.tight_layout()
        plt.title(f'Temperature on {str(min_day)}')
        plt.show()

### 1. min_age, max_age를 통해 2220, 1416 데이터를 타겟 데이터로 선정하였음

In [4]:
import pandas as pd

min_df = pd.read_csv('./data/senhub/hub_device_2220.csv')
max_df = pd.read_csv('./data/senhub/hub_device_1416.csv')

min_df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)
max_df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)

min_df['updated'] = pd.to_datetime(min_df['updated'])
max_df['updated'] = pd.to_datetime(max_df['updated'])

min_df.rename(columns={'device_id_x' : 'device_id'})
max_df.rename(columns={'device_id_x' : 'device_id'})

,updated,device_id,days,sex,temperature,humidity,voc,tem,hum
0,2023-05-30 22:28:20,1416.0,1520.0,1.0,3008.0,3911.0,0.0,NaN,NaN
1,2023-05-30 22:28:21,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2023-05-30 22:28:22,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2023-05-30 22:28:23,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2023-05-30 22:28:24,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7214980,2023-08-22 10:37:58,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214981,2023-08-22 10:37:59,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214982,2023-08-22 10:38:00,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214983,2023-08-22 10:38:01,1416.0,1604.0,1.0,2956.0,5401.0,0.0,2475.0,7496.0


- 겹치는 그래프가 없네...

### 2. 데이터 수량 많은 아기(357)

In [ ]:
df_357 = pd.read_csv('./data/senhub/hub_device_357.csv')
make_df(df_357)

- 시각화 그래프 함수 구현

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

def  make_df(_df):
    
    # 데이터 전처리
    _df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)
    _df['updated'] = pd.to_datetime(_df['updated'])
    _df.rename(columns={'device_id_x' : 'device_id'})
    
    # 날짜 리스트 설정
    day_list = _df['updated'].dt.date.unique()
    
    # 평균값 담을 객체 생성
    sensor_mean_list = [0] * 3
    hub_mean_list = [0] * 3

    # 날짜별 데이터 호출
    for day in day_list:
        day_df = _df[_df['updated'].dt.date == day]
        
        # 해당 날짜의 데이터가 없다면 continue를 사용하여 다음 반복으로 넘어갑니다.
        if day_df['temperature'].isna().all():
            continue
        
        # 센서 온도 있는 경우에만 뒤의 값으로 보간법 시행
        day_df['tem'] = day_df['tem'].bfill()
        day_df.loc[day_df['temperature'].isnull(), 'tem'] = None
        
        # 3일 평균 데이터 갱신
        sensor_mean_list.pop(0)
        sensor_mean_list.append(day_df['temperature'].mean())
        
        if not day_df['tem'].isna().all():
            hub_mean_list.pop(0)
            hub_mean_list.append(day_df['tem'].mean())
        
        # 3일 평균 데이터 가져오기
        sensor_not_zero = [i for i in sensor_mean_list if i!=0]
        hub_not_zero = [i for i in hub_mean_list if i!=0]
        
        sensor_mean = np.mean(sensor_not_zero)
        hub_mean = np.mean(hub_not_zero)
        
        # 평균 데이터 컬럼 추가
        day_df['sensor_mean'] = sensor_mean * len(day_df)
        day_df['hub_mean'] = hub_mean * len(day_df)
        
        # 이동평균 컬럼 추가
        day_df['temperature_moving_avg'] = day_df['temperature'].rolling(window=1200).mean()
        day_df['tem_moving_avg'] = day_df['tem'].rolling(window=1200).mean()
        
        _df.drop(['tem'], axis=1, inplace=True)
        day_df.drop(['days','sex','temperature','humidity','voc','hum'], axis=1)
        
        # 원본 데이터와 concat
        _df = pd.merge(_df, day_df, how='left', on=['device_id','updated'])
        
        break
    


### 1. min_age, max_age를 통해 2220, 1416 데이터를 타겟 데이터로 선정하였음

In [ ]:
import pandas as pd

min_df = pd.read_csv('./data/senhub/hub_device_2220.csv')
max_df = pd.read_csv('./data/senhub/hub_device_1416.csv')

min_df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)
max_df.drop(['_id_x', '_id_y','device_id_y'], axis=1, inplace=True)

min_df['updated'] = pd.to_datetime(min_df['updated'])
max_df['updated'] = pd.to_datetime(max_df['updated'])

min_df.rename(columns={'device_id_x' : 'device_id'})
max_df.rename(columns={'device_id_x' : 'device_id'})

,updated,device_id,days,sex,temperature,humidity,voc,tem,hum
0,2023-05-30 22:28:20,1416.0,1520.0,1.0,3008.0,3911.0,0.0,NaN,NaN
1,2023-05-30 22:28:21,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2023-05-30 22:28:22,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2023-05-30 22:28:23,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2023-05-30 22:28:24,1416.0,1520.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7214980,2023-08-22 10:37:58,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214981,2023-08-22 10:37:59,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214982,2023-08-22 10:38:00,1416.0,1604.0,1.0,NaN,NaN,NaN,NaN,NaN
7214983,2023-08-22 10:38:01,1416.0,1604.0,1.0,2956.0,5401.0,0.0,2475.0,7496.0


In [ ]:
df_357 = pd.read_csv('./data/senhub/hub_device_357.csv')
make_df(df_357)

C:\Users\yeong\AppData\Local\Temp\ipykernel_4892\3830756969.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['tem'] = day_df['tem'].bfill()
c:\Users\yeong\miniconda3\envs\monit\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\yeong\miniconda3\envs\monit\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\yeong\AppData\Local\Temp\ipykernel_4892\3830756969.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

KeyError: 'device_id'

In [ ]:
draw_plot(df_357)